# Digits recognition

In [1]:
import numpy as np
import pandas as pd
from sklearn import neural_network


data = pd.read_csv("Data/train.csv")
n, p = np.shape(data)

In [39]:
ntrain = 2500
ntest = 100
train = data.iloc[0:ntrain,1:]/256
trainlabel = data.iloc[0:ntrain, 0]
test = data.iloc[ntrain:ntest+ntrain,1:]/256
testlabel = data.iloc[ntrain:ntest+ntrain,0]

In [40]:
ytrain = np.zeros((ntrain, 10))
ytrain[(np.arange(ntrain),trainlabel.values)] = 1

Codons un neural net "from scratch":

In [41]:
class nnet:
    def __init__(self, sizes):
        self.nblayers = len(sizes)
        self.poids = [np.random.randn(i, j) for i, j in zip(sizes[0:], sizes[1:])]
        self.biais = [np.random.randn(1, i) for i in sizes[1:]]
    
    def activation(self, x):
        return 1/(1+np.exp(-x)) 
    
    def activationPrime(self, x):
        a = self.activation(x)
        return a*(1-a)
    
    def gradPerte(self, a, y):
        return a-y
    
    def feedforward(self, x):
        a = x
        for p,b in zip(self.poids, self.biais):
            a = self.activation(np.dot(a, p) + b)
        return a
        
    def backpropagation(self, x, y):
        n = len(x)
        a = [x.reshape(1,n)]
        z = []
        y = y.reshape(1,10)
        for p,b in zip(self.poids, self.biais):
            z.append(np.dot(a[-1], p) + b)
            a.append(self.activation(z[-1]))
        
        delta = [self.gradPerte(a[-1], y)*self.activationPrime(z[-1])]
        deriveePoids = [np.dot(a[-2].transpose(), delta[0])]
        
        for i in range(2, self.nblayers):
            delta = [np.dot(delta[0], self.poids[-i+1].transpose())*self.activationPrime(z[-i])] + delta
            deriveePoids = [np.dot(a[-i-1].transpose(), delta[0])] + deriveePoids
        
        return deriveePoids, delta
    
    def fit(self, x, y, batch_size=10, learning_rate = 3, max_iter = 30):
        n = len(x)
        indices = np.arange(n)
        
        for j in range(max_iter):
            np.random.shuffle(indices)

            for i in range(0, n, batch_size):
                sumP = [np.zeros(p.shape) for p in self.poids]
                sumB = [np.zeros(b.shape) for b in self.biais]

                for ind in indices[i:i+batch_size]:
                   
                    dp, db = self.backpropagation(x[ind],y[ind])
                    
                    
                    sumP = [p + dp1 for p, dp1 in zip(sumP, dp)]
                    sumB = [b + db1 for b, db1 in zip(sumB, db)]
                    
                    
                self.poids = [p - (learning_rate/batch_size)*dp1 for p, dp1 in zip(self.poids, sumP)]
                self.biais = [b - (learning_rate/batch_size)*db1 for b, db1 in zip(self.biais, sumB)]
                
    def predict(self, x):
        predictions = []
        for elm in x:
            predictions.append(np.argmax(self.feedforward(elm), axis=1))
        return np.array(predictions)
        
    def performance(self, x, y):
        pred = self.predict(x)
        n = len(x)
        
        return np.sum(y.reshape(1,n) == pred.reshape(1,n))/n

In [42]:
net = nnet([784,30,10])

In [43]:
net.fit(train.values, ytrain)

In [45]:
print("Le taux de bonnes prédictions est alors: {}%".format(net.performance(test.values, testlabel.values)*100))

Le taux de bonnes prédictions est alors: 89.0%
